In [1]:
#https://github.com/garywu/google-compute-engine-selenium
# # install chromium, its driver, and selenium
# !yes |sudo apt install chromium-chromedriver
# !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',options=options)
driver.quit()

# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results



In [2]:

from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [3]:
import csv


f = open("lager157.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","color","size","price","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("lager157.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,color,size,price,url,img_url,article_no,blob_url


In [4]:
!mkdir lager157

mkdir: cannot create directory ‘lager157’: File exists


In [5]:
from bs4 import BeautifulSoup
import time
import requests
categorys=['shorts','jeans','ytterkläder','funktionsplagg','training','toppar','tshirts','skjortor','klanning-kjol',
          'stickat','hoodar','trojor','byxor','underklader','swimwear','strumpor']
main_urls=['https://www.lager157.com/sv-se/artiklar/barn/shorts/',
          'https://www.lager157.com/sv-se/artiklar/barn/jeans/',
          'https://www.lager157.com/sv-se/artiklar/barn/ytterkläder/',
          'https://www.lager157.com/sv-se/artiklar/barn/funktionsplagg/',
          'https://www.lager157.com/sv-se/artiklar/barn/training/',
          'https://www.lager157.com/sv-se/artiklar/barn/toppar/',
          'https://www.lager157.com/sv-se/artiklar/barn/t-shirts/',
          'https://www.lager157.com/sv-se/artiklar/barn/skjortor/',
          'https://www.lager157.com/sv-se/artiklar/barn/klänning-kjol/',
          'https://www.lager157.com/sv-se/artiklar/barn/stickat/',
          'https://www.lager157.com/sv-se/artiklar/barn/hoodar/',
          'https://www.lager157.com/sv-se/artiklar/barn/tröjor/',
          'https://www.lager157.com/sv-se/artiklar/barn/byxor/',
          'https://www.lager157.com/sv-se/artiklar/barn/underkläder/',
          'https://www.lager157.com/sv-se/artiklar/barn/swimwear/',
          'https://www.lager157.com/sv-se/artiklar/barn/strumpor/']
for c,main_url in enumerate(main_urls):
    print(main_url)
    category=categorys[c]
    driver= webdriver.Chrome('chromedriver',options=options)
    driver.get(main_url)
    driver.set_window_size(1020,720)
    pause=4
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        products=driver.find_elements_by_xpath('//article[@class="ListArticle"]')
    print(len(products))
    for p in products:
        url=p.find_element_by_tag_name("a").get_attribute("href")
        html_doc=requests.get(url).content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')
        name=soup.find("span",attrs={"class":"name"}).text
        price=soup.find("span",attrs={"class":"price"}).text
        article_no=soup.find("span",attrs={"class":"product_id"}).text
        l1=soup.find("span",attrs={"class":"image_url"}).text
        l2=l1.split('/')
        img_url='https://www.lager157.com/media/'+l2[-1]
        
        ac='lager157'+'/'+article_no+str('.jpg')
#             opener = urllib.request.URLopener()
#             opener.addheader('User-Agent', 'whatever')
#             filename, headers2 = opener.retrieve(img_url, ac)

        rr = requests.get(img_url, allow_redirects=True)
        open(ac, 'wb').write(rr.content)

        blob_name=str('lager157')+str('/')+str(article_no)
        path_to_file='lager157/'+article_no+str('.jpg')
        blob_url=upload_to_bucket(blob_name, path_to_file)

        brand='lager157'
        gender='unisex'
        
        color=soup.find("span",attrs={"class":"color"}).text
        sizes=soup.find_all("span",attrs={"class":"size"})
        this_set=set()
        for sz in sizes:
            size2=sz.text
            this_set.add(size2)
        for size in this_set:
#             print(name,price,color,article_no,size)
            with open("lager157.csv", "a", encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([name,brand,gender,category,color,size,price,url,img_url,article_no])


https://www.lager157.com/sv-se/artiklar/barn/shorts/
35
https://www.lager157.com/sv-se/artiklar/barn/jeans/
20
https://www.lager157.com/sv-se/artiklar/barn/ytterkläder/
20
https://www.lager157.com/sv-se/artiklar/barn/funktionsplagg/
76
https://www.lager157.com/sv-se/artiklar/barn/training/
20
https://www.lager157.com/sv-se/artiklar/barn/toppar/
37
https://www.lager157.com/sv-se/artiklar/barn/t-shirts/
59
https://www.lager157.com/sv-se/artiklar/barn/skjortor/
19
https://www.lager157.com/sv-se/artiklar/barn/klänning-kjol/
21
https://www.lager157.com/sv-se/artiklar/barn/stickat/
23
https://www.lager157.com/sv-se/artiklar/barn/hoodar/
29
https://www.lager157.com/sv-se/artiklar/barn/tröjor/
29
https://www.lager157.com/sv-se/artiklar/barn/byxor/
52
https://www.lager157.com/sv-se/artiklar/barn/underkläder/
11
https://www.lager157.com/sv-se/artiklar/barn/swimwear/
20
https://www.lager157.com/sv-se/artiklar/barn/strumpor/
13


In [6]:
from bs4 import BeautifulSoup
for p in products:
    url=p.find_element_by_tag_name("a").get_attribute("href")
    html_doc=requests.get(url).content.decode()
    soup = BeautifulSoup(html_doc, 'html.parser')
    name=soup.find("span",attrs={"class":"name"}).text
    price=soup.find("span",attrs={"class":"price"}).text
    article_no=soup.find("span",attrs={"class":"product_id"}).text
    l1=soup.find("span",attrs={"class":"image_url"}).text
    l2=l1.split('/')
    img_url='https://www.lager157.com/media/'+l2[-1]
    color=soup.find("span",attrs={"class":"color"}).text
    sizes=soup.find_all("span",attrs={"class":"size"})
    this_set=set()
    for sz in sizes:
        size=sz.text
        this_set.add(size)
    for size in this_set:
        print(name,price,color,article_no,size)
    

Strumpor "Raggsockar" 30.0 Grey 10141 25-30
Strumpor "Raggsockar" 30.0 Grey 10141 31-35
Strumpor "Raggsockar" 30.0 Grey 10141 20-24
Strumpor "Coloured socks" 20.0 Combo 9 13061 41-45
Strumpor "Coloured socks" 20.0 Combo 9 13061 36-40
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 25-30
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 31-35
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 20-24
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 25-30
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 31-35
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 20-24
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 25-30
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 31-35
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 20-24
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 25-30
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 31-35
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 20-24
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 25-30
Strumpor Basic kids "5-pack" 60.0 Combo 8 12170 31-35
Strumpor B

In [7]:
url

'https://www.lager157.com/sv-se/article/strumpor-baby-sock?attr1_id=49'

In [8]:
this_set

{'1', '2'}

In [9]:
l1=soup.find("span",attrs={"class":"image_url"}).text
l2=l1.split('/')
img_url='https://www.lager157.com/media/'+l2[-1]

In [10]:
img_url

'https://www.lager157.com/media/SOCK-BEBE-SOCK_11736_COMBO-1_FRONT.jpg'

In [11]:
'https://www.lager157.com/media'+img_url

'https://www.lager157.com/mediahttps://www.lager157.com/media/SOCK-BEBE-SOCK_11736_COMBO-1_FRONT.jpg'

In [12]:
url

'https://www.lager157.com/sv-se/article/strumpor-baby-sock?attr1_id=49'